In [ ]:
!pip install tensor-dash

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
dataset = pd.read_csv('/content/drive/MyDrive/Agro/plant/train.csv')

In [24]:
!pip install tensorflow
!pip install streamlit
!pip install matplotlib
!pip install pandas
!pip install scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.3 MB/s eta 0:00:00


In [29]:
# Import necessary libraries
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to files
train_csv_path = '/content/drive/MyDrive/Agro/plant/train.csv'
test_csv_path = '/content/drive/MyDrive/Agro/plant/test.csv'
image_folder = '/content/drive/MyDrive/Agro/plant/images/'  # Update this if the images are in a specific folder

# Load the train.csv and test.csv
train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)

# Preview the dataset
print("Train Data:")
print(train_df.head())

# Splitting the training data into train and validation sets
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

print(f"Training samples: {len(train_data)}, Validation samples: {len(val_data)}")


Train Data:
  image_id  healthy  multiple_diseases  rust  scab
0  Train_0        0                  0     0     1
1  Train_1        0                  1     0     0
2  Train_2        1                  0     0     0
3  Train_3        0                  0     1     0
4  Train_4        1                  0     0     0
Training samples: 1456, Validation samples: 365


In [31]:
import os

# List all image filenames in the folder
image_filenames = set(os.listdir(image_folder))

# Check for mismatched filenames
mismatched_filenames = [img for img in train_df['image_id'] if img not in image_filenames]

if mismatched_filenames:
    print(f"Number of mismatched filenames: {len(mismatched_filenames)}")
    print("Examples of mismatched filenames:", mismatched_filenames[:10])
else:
    print("All filenames match!")


Number of mismatched filenames: 1821
Examples of mismatched filenames: ['Train_0', 'Train_1', 'Train_2', 'Train_3', 'Train_4', 'Train_5', 'Train_6', 'Train_7', 'Train_8', 'Train_9']


In [33]:
import os

# List the first few filenames to check their extensions
image_folder = '/content/drive/MyDrive/Agro/plant/images/'  # Update this path if necessary
example_files = os.listdir(image_folder)[:10]
print("Example filenames in the folder:", example_files)


Example filenames in the folder: ['Test_648.jpg', 'Train_1308.jpg', 'Test_593.jpg', 'Test_1760.jpg', 'Test_1803.jpg', 'Test_1148.jpg', 'Test_115.jpg', 'Test_1771.jpg', 'Test_1766.jpg', 'Train_1260.jpg']


In [34]:
# Add '.jpg' extension to all filenames in train and val data
train_data['image_id'] = train_data['image_id'].apply(lambda x: f"{x}.jpg")
val_data['image_id'] = val_data['image_id'].apply(lambda x: f"{x}.jpg")

# Check updated filenames
print("Updated train data image_id examples:", train_data['image_id'].head())


Updated train data image_id examples: 1455    Train_1455.jpg
1552    Train_1552.jpg
300      Train_300.jpg
1708    Train_1708.jpg
611      Train_611.jpg
Name: image_id, dtype: object


In [36]:
import os

# Define the image folder path
image_folder = '/content/drive/MyDrive/Agro/plant/images/'  # Adjust if needed

# Get a set of all filenames in the folder
image_filenames = set(os.listdir(image_folder))

# Check for missing files
missing_files = [img for img in train_data['image_id'] if img not in image_filenames]
if missing_files:
    print(f"Number of missing files: {len(missing_files)}")
    print("Examples of missing files:", missing_files[:10])
else:
    print("All filenames match!")


All filenames match!


In [38]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the data generator with rescaling and validation split
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Define training generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='/content/drive/MyDrive/Agro/plant/images/',
    x_col='image_id',
    y_col=['healthy', 'multiple_diseases', 'rust', 'scab'],
    target_size=(224, 224),  # You can adjust this size if needed
    batch_size=32,
    class_mode='raw',
    subset='training'  # Training subset
)

# Define validation generator
val_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='/content/drive/MyDrive/Agro/plant/images/',
    x_col='image_id',
    y_col=['healthy', 'multiple_diseases', 'rust', 'scab'],
    target_size=(224, 224),  # Same size as training
    batch_size=32,
    class_mode='raw',
    subset='validation'  # Validation subset
)


Found 1165 validated image filenames.
Found 291 validated image filenames.


In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 classes: healthy, multiple_diseases, rust, scab
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()  # Print the model architecture


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)                   │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 186624)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 128)                 │      23,888,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,907,908 (91.20 MB)

 Trainable params: 23,907,908 (91.20 MB)

 Non-trainable params: 0 (0.00 B)

In [42]:
steps_per_epoch = train_generator.n // train_generator.batch_size
validation_steps = val_generator.n // val_generator.batch_size

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)


Epoch 1/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 146s 4s/step - accuracy: 0.3352 - loss: 1.2713 - val_accuracy: 0.3403 - val_loss: 1.2955
Epoch 2/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4062 - loss: 1.0987 - val_accuracy: 0.0000e+00 - val_loss: 1.3062
Epoch 3/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 143s 4s/step - accuracy: 0.4330 - loss: 1.1994 - val_accuracy: 0.3611 - val_loss: 1.2978
Epoch 4/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 142ms/step - accuracy: 0.4688 - loss: 1.0909 - val_accuracy: 0.0000e+00 - val_loss: 1.5145
Epoch 5/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 152s 4s/step - accuracy: 0.4752 - loss: 1.1811 - val_accuracy: 0.4549 - val_loss: 1.2412
Epoch 6/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 156ms/step - accuracy: 0.5000 - loss: 1.0849 - val_accuracy: 0.0000e+00 - val_loss: 1.2130
Epoch 7/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 188s 4s/step - accuracy: 0.5085 - loss: 1.1039 - val_accuracy: 0.4653 - val_loss: 1.1963
Epoch 8/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 144ms/step - accuracy: 0.3750 - loss: 1.0801 - val_accura